#### Standalone cppyy (C++)
 * Awkward Arrays can be passed to a C++ (possibly templated) function defined by cppyy compiler
 *  A user does not need to know what **cpp_type** is
    * **cpp_type** is generated on demand when the Array needs to be passed to the function
 * Based on cppyy from a master branch (3.1.0)
 
 Note: ROOT and standalone cppyy cannot be used in the same environment

In [1]:
import awkward as ak

In [2]:
ak.__version__

'2.4.5'

In [3]:
import cppyy

In [4]:
cppyy.__version__

'3.0.0'

In [5]:
cppyy.__version__="3.0.1"

In [6]:
cppyy.__version__

'3.0.1'

In [7]:
array = ak.Array(
  [
    [{"x": 1, "y": [1.1]}, {"x": 2, "y": [2.2, 0.2]}],
    [],
    [{"x": 3, "y": [3.0, 0.3, 3.3]}],
  ]
)

source_code_cpp = """
template<typename T>
double go_fast_cpp(T& awkward_array) {
 double out = 0.0;
 
 for (auto list : awkward_array) {
   for (auto record : list) {
     for (auto item : record.y()) {
       out += item;
     }
   }
 }

 return out;
}
"""
cppyy.cppdef(source_code_cpp)

out = cppyy.gbl.go_fast_cpp[array.cpp_type](array)
assert out == ak.sum(array["y"])


In [8]:
out

10.1

In [9]:
%%timeit
cppyy.gbl.go_fast_cpp[array.cpp_type](array)

5.97 µs ± 41.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [10]:
%%timeit
ak.sum(array["y"])

184 µs ± 6.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
%%timeit
out = 0.0
for list in array:
    for record in list["y"]:
        for item in record:
            out += item 

307 µs ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
out

10.1